# Important note!

Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your GT login and the GT logins of any of your collaborators below. (The GT logins are worth 1 point per notebook, so don't miss the opportunity to get a free point!)

In [ ]:
YOUR_ID = "" # Please enter your GT login, e.g., "rvuduc3" or "gtg911x"
COLLABORATORS = [] # list of strings of your collaborators' IDs

In [ ]:
import re

RE_CHECK_ID = re.compile (r'''[a-zA-Z]+\d+|[gG][tT][gG]\d+[a-zA-Z]''')
assert RE_CHECK_ID.match (YOUR_ID) is not None

collab_check = [RE_CHECK_ID.match (i) is not None for i in COLLABORATORS]
assert all (collab_check)

del collab_check
del RE_CHECK_ID
del re

**Jupyter / IPython version check.** The following code cell verifies that you are using the correct version of Jupyter/IPython.

In [ ]:
import IPython
assert IPython.version_info[0] >= 3, "Your version of IPython is too old, please update it."

# Infection via the continuous SIR-PDE model

The [O'Leary (2008) reading](https://t-square.gatech.edu/access/content/group/gtc-239f-fc11-5690-9dae-2dc96b59f372/OLeary-2008-sccs--infection.pdf) presents the following version of a continuous partial differential equation (PDE)-based version of the susceptible-infected-recovered (SIR) model in two spatial dimensions, where $S = S(x, y, t)$, $I = I(x, y, t)$, and $R = R(x, y, t)$.

$$
\begin{eqnarray}
  \dfrac{\partial S}{\partial t}
  & = & -\tau_0 \cdot I \cdot S - \delta_0 \cdot \left( \dfrac{\partial^2 I}{\partial x^2} + \dfrac{\partial^2 I}{\partial y^2} \right) \cdot S \\
  \dfrac{\partial I}{\partial t}
  & = & \tau_0 \cdot I \cdot S + \delta_0 \cdot \left( \dfrac{\partial^2 I}{\partial x^2} + \dfrac{\partial^2 I}{\partial y^2} \right) \cdot S - \dfrac{I}{\kappa_0} \\
  \dfrac{\partial R}{\partial t}
  & = & \dfrac{I}{\kappa_0}
\end{eqnarray}
$$

In vector form,

$$
  \dfrac{\partial}{\partial t} \vec{y}(t)
  \equiv
  \dfrac{\partial}{\partial t} \left( \begin{matrix} S \\ I \\ R \end{matrix} \right)
  =
  \left( \begin{matrix}
    -\tau_0 \cdot I \cdot S - \delta_0 \cdot \left( \dfrac{\partial^2 I}{\partial x^2} + \dfrac{\partial^2 I}{\partial y^2} \right) \cdot S \\
    \tau_0 \cdot I \cdot S + \delta_0 \cdot \left( \dfrac{\partial^2 I}{\partial x^2} + \dfrac{\partial^2 I}{\partial y^2} \right) \cdot S - \dfrac{I}{\kappa_0} \\
    \dfrac{I}{\kappa_0}
  \end{matrix} \right)
  \equiv
  \vec{f}(\vec{y}).
$$

Per our usual convention, we'll refer to $\vec{f}(\vec{y})$ as the "right-hand side" of this model.

If the population density is uniform, then we can choose $S(x, y, t)$, $I(x, y, t)$, and $R(x, y, t)$ to be concentrations (fractions per unit area) such that

$$
  \int_{\Omega} \left[ S(x, y, t) + I(x, y, t) + R(x, y, t) \right] \, dx \, dy = 1.
$$

In this notebook, you will implement a simulator for this model, which we'll refer to as the _continous (space-time) SIR system_.

## Setup

To reuse an ODE solver (e.g., `odeint()`), we need to eliminate all but one independent variable. Let's do so by discretizing the spatial variables, $x$ and $y$.

Suppose the spatial "world" is a square domain with $0 \leq x, y \leq \lambda_0$. Let's represent the world instead on a finite $n \times n$ domain at a discrete set of equally-spaced discrete points, $\{(x_i, y_j) \,|\, 0 \leq i, j \leq n\}$, where $x_i \equiv i \Delta x$, $y_j \equiv j \Delta y$, and $\Delta x = \Delta y = \frac{\lambda_0}{n-1}$.

In [ ]:
import numpy as np

LAMBDA_0_DEMO = 2.0
N_DEMO = 5
DX_DEMO = LAMBDA_0_DEMO / (N_DEMO-1)

print ("Domain: 0 <= x, y <= {}".format (LAMBDA_0_DEMO))
print ("Finite representation: {} x {}".format (N_DEMO, N_DEMO))
print ("Grid spacing: {} x {}".format (DX_DEMO, DX_DEMO))

points_demo = np.linspace (0, LAMBDA_0_DEMO, N_DEMO)
print ("{{x_i}} = {{y_j}} = {}".format (set (points_demo)))

**"World" abstraction.** With this discretization, $\left\{ \hat{S}_{ij}(t) \equiv S(x_i, y_j, t) \right\}$, is the concentration of susceptible individuals at position $(x_i, y_j)$ at time $t$. Similarly, we can define $\left\{ \hat{I}_{ij}(t) \right\}$ and $\left\{ \hat{R}_{ij}(t) \right\}$ for infected and recovered populations as well. Furthermore, we will assume that

$$
\begin{eqnarray}
  \sum_{i=0}^{n-1} \sum_{j=0}^{n-1} \left[ \hat{S}_{ij}(t) + \hat{I}_{ij}(t) + \hat{R}_{ij}(t) \right] \cdot \Delta x \cdot \Delta y & \approx & 1.
\end{eqnarray}
$$

At a given time $t$, let's store the state variables in a 3-D NumPy array, `W[:, :, :]`, where

* `W[0, i, j]` is the `n`-by-`n` NumPy array corresponding to the susceptible concentration, $\hat{S}_{ij}(t)$;
* `W[1, i, j]` for $\hat{I}_{ij}(t)$; and
* `W[2, i, j]` for $\hat{R}_{ij}(t)$.

This definition motivates the following functions.

In [ ]:
def susceptible (world):
    """Returns the susceptible concentration of a world."""
    return world[0, :, :]

def infected (world):
    """Returns the infected concentration of a world."""
    return world[1, :, :]

def recovered (world):
    """Returns the recovered concentration of a world."""
    return world[2, :, :]

def len_dim (world):
    """Returns the number of discrete cells along one dimension of a world."""
    assert world.ndim == 3
    assert world.shape[1] == world.shape[2]
    return world.shape[1]

**Exercise 1** (3 points). Complete the following function, which should create a new discretized world of size $n \times n$ to represent a domain on $[0, \lambda_0] \times [0, \lambda_0]$.

This world should be returned as a 3-D NumPy array, `W[:, :, :]`, where `W[0, i, j]` is the $n \times n$ grid rerepsenting the susceptible concentration in the immediate neighborhood of `(i, j)`; and, similarly, `W[1, :, :]` and `W[2, :, :]` representing the infected and recovered concentrations, respectively.

The concentrations at the approximate center should be fully infected; everywhere else, the concentrations should be fully susceptible.

> _Note:_ Recall that the sum of concentrations, multipled by the area $\Delta x \Delta y$, should equal 1.0.

In [ ]:
def create_world (lambda_0, n):
    """Returns a new world with a pocket of infection in the center."""
    W = np.zeros ((3, n, n))
    # YOUR CODE HERE
    raise NotImplementedError()
    return W

def print_world (world):
    print (world[:, ::-1, ::-1])

In [ ]:
W_demo = create_world (LAMBDA_0_DEMO, N_DEMO)
print ("World: {} x {}, length lambda_0 = {} on each side.\n".format (N_DEMO, N_DEMO, LAMBDA_0_DEMO))
print_world (W_demo)
assert W_demo.shape == (3, N_DEMO, N_DEMO)
assert np.allclose (susceptible (W_demo) + infected (W_demo), 0.16, atol=1e-3)
assert np.isclose (infected (W_demo)[2, 2], 0.16, atol=1e-3)
assert np.isclose (np.sum (infected (W_demo)), 0.16, atol=1e-3)
assert np.allclose (recovered (W_demo), 0.0, atol=1e-15)

W_demo_even = create_world (4.0, 4)
print ("\nWorld (even): {} x {}, length lambda_0 = {} on each side.\n".format (4, 4, LAMBDA_0_DEMO))
print_world (W_demo_even)
assert W_demo_even.shape == (3, 4, 4)
assert np.allclose (susceptible (W_demo_even) + infected (W_demo_even), 0.03515625, atol=1e-8)
assert np.allclose (infected (W_demo_even)[1:3, 1:3], 0.03515625, atol=1e-8)
assert np.allclose (recovered (W_demo_even), 0.0, atol=1e-15)

print ("\n(Passed.)")

**Pretty printing.** Here is some additional support code to draw a picture of the state of the world.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def show_grid (G,
               xticks=None, xlabels=None,
               yticks=None, ylabels=None,
               vticks=None, vlabels=None):
    """Displays a checkerboard plot of the values in a 2-D NumPy array."""
    m, n = G.shape
    
    if vticks is None:
        vticks = np.linspace (0, 1, 11)

    plt.pcolor (G, vmin=min (vticks), vmax=max (vticks), edgecolor='black')
    if m == n: plt.axis ('square')
    plt.axis ([0, m, 0, n])
    
    if xticks is not None:
        if xlabels is not None:
            plt.xticks (xticks, xlabels)
        else:
            plt.xticks (xticks)
            
    if yticks is not None:
        if ylabels is not None:
            plt.yticks (yticks, ylabels)
        else:
            plt.yticks (yticks)

    cb = plt.colorbar ()
    cb.set_ticks (vticks)
    if vlabels:
        cb.set_ticklabels (vlabels)
        
def show_world (W, lambda_0=None, t=None):
    assert W.ndim == 3
    assert W.shape[1] == W.shape[2] # Square worlds only
    
    n = W.shape[1]
    nticks = 5
    dn = int (n/nticks)
    
    if lambda_0 is None:
        lambda_0 = 1.0

    dx = lambda_0 / (n-1)
    xticks = np.arange (0, dn*nticks + 1, dn)
    xlabels = xticks*dx
    
    if t is None:
        title_args = ''
    else:
        title_args = ' (t={})'.format (t)
        
    W_plot = W * (dx*dx)
    vticks = np.linspace (0, np.max (W_plot), 11)
    args = {'xticks': xticks+dn/2, 'xlabels': xlabels,
            'yticks': xticks+dn/2, 'ylabels': xlabels,
            'vticks': vticks}
    plt.subplot (1, 3, 1)
    show_grid (susceptible (W_plot), **args)
    plt.title ('Susceptible' + title_args)
    plt.subplot (1, 3, 2)
    show_grid (infected (W_plot), **args)
    plt.title ('Infected' + title_args)
    plt.subplot (1, 3, 3)
    show_grid (recovered (W_plot), **args)
    plt.title ('Recovered' + title_args)
    
plt.figure (figsize=(15, 5))
show_world (W_demo, LAMBDA_0_DEMO)

**Exercise 2** (2 points). Recall that the sum of the concentrations should integrate to 1. Implement a function that, given the discretized concentrations, approximates that integral.

In [ ]:
def integrate_domain (W, lambda_0=1.0):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
integral_demo = integrate_domain (W_demo, lambda_0=LAMBDA_0_DEMO)
print ("Integral:", integral_demo)
assert np.isclose (integral_demo, 1.0, 1e-15*N_DEMO*N_DEMO)
print ("\n(Passed.)")

**Centered finite differences.** Recall that in the discretized domain we may estimate the second derivative of a function $f(x)$ by the centered finite difference approximation,

$$
  \frac{d^2 f(x)}{d x^2} \approx \dfrac{f(x - \Delta x) - 2 f(x) + f(x + \Delta x)}{(\Delta x)^2}.
$$

Thus, if we create state variables $f_i \equiv f(x_i)$ at a set of discrete points $\{x_i\}$, then

$$
  \left. \frac{d^2 f(x)}{d x^2} \right|_{x=x_i}
     \approx \dfrac{f_{i-1} - 2 f_i + f_{i+1}}{(\Delta x)^2}.
$$

**Exercise 3** (3 points). Given a 2-D grid of values, `G[:, :]`, complete the function `diffuse2d(G, delta_0, dx)`, that computes the centered finite difference approximation of the second derivative in each direction, $x$ and $y$.

That is, suppose `G[i, j]` is the discretized value of a function $g_{i,j} \equiv g(x_i, y_j)$; then `diffuse2d(G, delta_0, dx)` should compute the 2-D centered finite difference approximation,

$$
\begin{eqnarray}
  \left. \delta_0 \left(\frac{\partial^2}{\partial x^2} + \frac{\partial^2}{\partial y^2}\right) g(x, y) \right|_{x=x_i, y=y_j}
     & \approx & \delta_0 \left(
       \dfrac{g_{i-1,j} - 2 g_{i, j} + g_{i+1, j}}{(\Delta x)^2}
       + \dfrac{g_{i,j-1} - 2 g_{i, j} + g_{i, j+1}}{(\Delta y)^2}
     \right) \\
     & = & \dfrac{\delta_0}{(\Delta x)^2} \left( g_{i-1, j} + g_{i, j-1} - 4 g_{i, j} + g_{i+1, j} + g_{i, j+1} \right),
\end{eqnarray}
$$

assuming uniform step sizes $\Delta x = \Delta y$ and a square $n \times n$ grid. At the boundaries, assume infinte walls, e.g., $g_{-1, j} = g_{0, j}$, $g_{n, j} = g_{n-1, j}$, $g_{i, -1} = g_{i, 0}$, and $g_{i, n} = g_{i, n-1}$.

> _Note:_ The scaffolding below creates a variable, `d2G[:, :]`, to hold and return this approximation. The parameter `delta_0` corresponds to the diffusion coefficient $\delta_0$; the parameter `dx` corresponds to the grid spacing $\Delta x = \Delta y$. The scaffolding applies the scaling $\frac{\delta_0}{\Delta x^2}$ to `d2G` just before returning, so apply the finite differences accordingly.

In [ ]:
def diffuse2d (G, delta_0=1.0, dx=1.0):
    assert G.ndim == 2
    d2G = np.zeros (G.shape)
    
    # YOUR CODE HERE
    raise NotImplementedError()
    
    # Scale factor
    d2G *= delta_0 / (dx*dx)
    return d2G

In [ ]:
dI = diffuse2d (infected (W_demo), delta_0=0.01, dx=0.2)
print (dI)
assert np.allclose (dI[1:4, 1:4],
                    np.array ([[0., 0.04, 0.],
                               [0.04, -0.16, 0.04],
                               [0., 0.04, 0.]]),
                    atol=1e-3)
assert np.isclose (np.sum (dI), 0.0, atol=1e-15*(len_dim(W_demo)**2))
print ("\n(Passed.)")

**Exercise 4** (2 points). Given the current state of the world, `W[:, :, :]`, complete a function `f_pde2d(W, lambda_0, tau_0, kappa_0, delta_0)` that implements the approximate right-hand side of the continuous SIR system.

In [ ]:
def f_pde2d (W, lambda_0=1.0, tau_0=1.0, kappa_0=1.0, delta_0=1.0):
    dW = np.zeros (W.shape)

    # YOUR CODE HERE
    raise NotImplementedError()
    
    return dW

In [ ]:
TAU_0_DEMO = 0.8
KAPPA_0_DEMO = 4.0
DELTA_0_DEMO = 0.2

dW_demo = f_pde2d (W_demo,
                   lambda_0=LAMBDA_0_DEMO,
                   tau_0=TAU_0_DEMO,
                   kappa_0=KAPPA_0_DEMO,
                   delta_0=DELTA_0_DEMO)
print (dW_demo)

assert np.allclose (susceptible (dW_demo), np.array ([[0, 0, 0, 0, 0],
                                                      [0, 0, -.02048, 0, 0],
                                                      [0, -.02048, 0, -.02048, 0],
                                                      [0, 0, -.02048, 0, 0],
                                                      [0, 0, 0, 0, 0]]),
                    atol=1e-5)
assert np.allclose (infected (dW_demo), np.array ([[0, 0, 0, 0, 0],
                                                   [0, 0, .02048, 0, 0],
                                                   [0, .02048, -0.04, .02048, 0],
                                                   [0, 0, .02048, 0, 0],
                                                   [0, 0, 0, 0, 0]]),
                    atol=1e-5)
assert np.allclose (recovered (dW_demo), np.array ([[0, 0, 0, 0, 0],
                                                    [0, 0, 0, 0, 0],
                                                    [0, 0, 0.04, 0, 0],
                                                    [0, 0, 0, 0, 0],
                                                    [0, 0, 0, 0, 0]]),
                    atol=1e-5)
assert np.isclose (np.sum (dW_demo), 0, atol=1e-5)
print ("\n(Passed.)")

**Exercise 5** (2 points). Recall that the black-box ODE solver, `odeint(f, y0, ...)`, expects a the function `f()` to return a _vector_, i.e., a 1-D NumPy array. (Similarly, the initial condition `y0` must also be a vector.) However, our representation of the world is a 3-D NumPy array.

Implement a pair of functions, `y = world_to_vec(W)` and `W = vec_to_world(y)`, that can convert from the world abstract to a vector and back again.

> _Hint:_ Consider using NumPy's [`reshape()`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.reshape.html#numpy.reshape) function. You may assume that the world has shape `W[:3, :n, :n]`.

In [ ]:
def world_to_vec (W):
    assert W.ndim == 3 and W.shape[0] == 3 and W.shape[1] == W.shape[2]
    # YOUR CODE HERE
    raise NotImplementedError()
    
def vec_to_world (v):
    assert v.ndim == 1 and (v.shape[0] % 3) == 0
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
v_demo = world_to_vec (W_demo)
assert v_demo.shape == (3*N_DEMO*N_DEMO,)
V_demo = vec_to_world (v_demo)
assert W_demo.shape == V_demo.shape
assert (W_demo == V_demo).all ()
print ("\n(Passed.)")

**Exercise 6** (1 point). Lastly, recall that [`odeint(f, ...)`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.odeint.html#scipy.integrate.odeint) requires that `f()` be a callback function with the signature,

```python
def f (y, t, *args):
    ...
```

where `y[:]` is a vector (1-D NumPy array). Implement a right-hand side function, `f_pde2d_vec()` that is compatible with this interface.

> _Note:_ The testing code cell performs a simulation using `odeint()` and your `f_pde2d_vec()` at various time steps. It then creates an interactive widget you can use to draw the grids corresponding to the result. (The slider corresponds to a time step index.)

In [ ]:
def f_pde2d_vec (v, t, *args):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
import scipy as sp
from scipy.integrate import odeint

# Test cell for `f_pde2d_vec()`
ARGS_DEMO = (LAMBDA_0_DEMO, TAU_0_DEMO, KAPPA_0_DEMO, DELTA_0_DEMO)
T_MAX_DEMO = 25
NT_DEMO = 101
T_demo = np.linspace (0, 25, NT_DEMO)
y_demo = odeint (f_pde2d_vec, world_to_vec (W_demo), T_demo, args=ARGS_DEMO)
Y_demo = y_demo.reshape ((len (T_demo), 3, N_DEMO, N_DEMO))

def view_demo (i=0):
    W = Y_demo[i, :, :, :]
    plt.figure (figsize=(15, 5))
    show_world (W, LAMBDA_0_DEMO, t=T_demo[i])
    
from ipywidgets import interact
interact (view_demo, i=(0, NT_DEMO-1, max (1, int (NT_DEMO/20))))

In [ ]:
# Static output of the final result:
view_demo (NT_DEMO-1)